In [5]:
from langchain_community.document_loaders import PyMuPDFLoader, UnstructuredExcelLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
    AnnSearchRequest, RRFRanker, WeightedRanker,
    db
)
import numpy as np
from FlagEmbedding import BGEM3FlagModel
from transformers import pipeline
import os
from datetime import datetime
import torch
from tqdm import tqdm
import argparse

from milvusdb import VectorDB

devices = ["cuda:1"]
MILVUS_HOST = "localhost"
MILVUS_PORT = "19530"
TARGET_DB = "test_db"  # 自定义数据库名称
COLLECTION_NAME="hybrid_search_collection"
EMBEDDING_MODEL_PATH = "/disk3/lsp/models/BAAI/bge-m3"
DIM = 1024

# 连接 Milvus
connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)

# 创建数据库（如果不存在）
if TARGET_DB not in db.list_database():
    print(f"数据库 {TARGET_DB} 不存在，创建数据库")
    db.create_database(TARGET_DB)
# 删除数据库
# db.drop_database(TARGET_DB)

# 切换当前数据库
db.using_database(TARGET_DB)
print("当前数据库：", TARGET_DB)
# 列出当前所有collection
print("当前数据库集合：", utility.list_collections())

def drop_collection(collection_name):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)
        print(f"集合 {collection_name} 已删除")
    # for collection_name in utility.list_collections():
    #     utility.drop_collection(collection_name)
    #     print(f"Collection {collection_name} dropped.")

/disk3/lsp/miniconda3/envs/milvus/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


当前数据库： test_db
当前数据库集合： ['hybrid_search_collection']


In [ ]:
# EXCEL文本解析与处理
def extract_text_from_excel(excel_path):
    print(f"解析 Excel ：{excel_path}")
    loader = UnstructuredExcelLoader(excel_path)
    data = loader.load()
    return data

In [ ]:
from langchain_community.document_loaders import UnstructuredExcelLoader

loader = UnstructuredExcelLoader("/disk3/lsp/milvus_test/excel/浙江一分一档表19-22.xlsx", mode="elements")
docs = loader.load()
print(docs[0])

In [ ]:
import pandas as pd
from langchain_core.documents import Document
# 读取excel文件
df = pd.read_excel("/disk3/lsp/milvus_test/excel/浙江一分一档表19-22.xlsx")
data = df.to_string(index=False)
docs = [Document(page_content=data, metadata={"file_path": "/disk3/lsp/milvus_test/excel/浙江一分一档表19-22.xlsx"})]
print(docs[0].page_content)

In [34]:
# 文本分割
def split_documents(docs, chunk_size=1024, chunk_overlap=20):
    """使用LangChain的递归字符分割器"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", "。", "！", "？", "；"]
    )
    
    return text_splitter.split_documents(docs)

chunks = split_documents(docs)
texts = [doc.page_content for doc in chunks]
# print(texts[0])
# 提取纯列名行（第一行）
header_line = data.split('\n')[0] + '\n'  # 带换行符
texts = [header_line + text for text in texts]
print(texts[41])

省份   年份 科类  分数  本段人数   累计人数
浙江 2020 综合 715   352    352
浙江 2020 综合 714   352    352
浙江 2020 综合 713   352    352
浙江 2020 综合 712   352    352
浙江 2020 综合 711   352    352
浙江 2020 综合 710   352    352
浙江 2020 综合 709   352    352
浙江 2020 综合 708   352    352
浙江 2020 综合 707   352    352
浙江 2020 综合 706   352    352
浙江 2020 综合 705   352    352
浙江 2020 综合 704   352    352
浙江 2020 综合 703   352    352
浙江 2020 综合 702   352    352
浙江 2020 综合 701   352    352
浙江 2020 综合 700   352    352
浙江 2020 综合 699   352    352
浙江 2020 综合 698   352    352
浙江 2020 综合 697   352    352
浙江 2020 综合 696   352    352
浙江 2020 综合 695   352    352
浙江 2020 综合 694   352    352
浙江 2020 综合 693   352    352
浙江 2020 综合 692   352    352
浙江 2020 综合 691   352    352
浙江 2021 综合 750   331    331
浙江 2021 综合 749   331    331
浙江 2021 综合 748   331    331
浙江 2021 综合 747   331    331
浙江 2021 综合 746   331    331
浙江 2021 综合 745   331    331
浙江 2021 综合 744   331    331
浙江 2021 综合 743   331    331
浙江 2021 综合 742   331    331
浙江 2021 综合 741   331

In [12]:
import pandas as pd
from langchain_core.documents import Document

# 读取Excel并跳过"序号"列
file_path = "/disk3/lsp/milvus_test/excel/浙江一分一档表19-22.xlsx"
df = pd.read_excel(file_path)  # 修改"序号"为实际列名

# 方法2：逐行生成多个Document（推荐用于检索场景）
docs_multiple = []
for _, row in df.iterrows():
    content = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    docs_multiple.append(Document(
        page_content=content,
        metadata={"file_path": file_path, "columns": list(df.columns)}
    ))

print(docs_multiple[0].page_content)
print("多文档数量：", len(docs_multiple))

省份: 浙江
年份: 2019
科类: 综合
分数: 695
本段人数: 32
累计人数: 330
多文档数量： 3000
